# Install requirements

`pip install -r requirements.txt`

# Data set up 

## Load existing dataset

In [1]:
import pandas as pd
df = pd.read_csv('tournesol_scores_above_20_2024-03-27.csv')

## Create dataset

Download videos scores

In [1]:
import io
import zipfile

import requests
import pandas as pd

#Download tournesol scores
response = requests.get("https://api.tournesol.app/exports/all")
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
collective_scores = pd.read_csv(zip_file.open("collective_criteria_scores.csv"))

#Requests youtube metadata : publication date, title and channel
from dataset import build_dataset
df = build_dataset(collective_scores) #only keeps videos with a tournesol score above 20

#save dataframe
import datetime
df.to_csv('tournesol_scores_above_20_'+datetime.date.today().isoformat()+'.csv')

# Tests set up

In [2]:
#Test parameters
bundle_size = 4
recent_videos_proportion = 0.75

recent_videos_to_sample = int(bundle_size * recent_videos_proportion)
old_videos_to_sample = int(bundle_size * (1 - recent_videos_proportion))

recent_videos_max_age_in_days = 21

#Computes the age of each video
import datetime

from utils import get_age_in_days, construct_L_Ensemble

today = datetime.datetime.today()
df['age_in_days'] = df.apply(lambda x: get_age_in_days(x, today), axis="columns")

#Construct L-Ensemble
power = 2.8 #Model parameters
discount = 7.3
caracteristic_time = 31

dpp = construct_L_Ensemble(df, power, discount, caracteristic_time)

#load preferences results
try:
    preferences_results_series = pd.read_csv('preferences_results.csv')
except FileNotFoundError:
    preferences_results_series = preferences_results_series = pd.Series(data={'dpp':0, 'uniform':0}) 

L = Phi.T Phi was computed: Phi (dxN) with d>=N


# Sample and display bundles

In [5]:
#Display bundles
from widget_layout import construct_bundles_widget
construct_bundles_widget(df, dpp, preferences_results_series, recent_videos_to_sample, old_videos_to_sample, bundle_size)


In [7]:
preferences_results_series

dpp        2
uniform    2
dtype: int64

In [ ]:
#Save preferences
preferences_results_series.to_csv('preferences_results.csv')